# Design new benchmarks
In order to design new benchmarks in the context of power networks, a configuration file should be created from which the benchmark is initiated. The LIPS platform via `ConfigManager` class ease this operation, and this notebook shows how to use its different functionalities.

## TOC:
* [Read an existing config](#first-bullet)
* [Create a new config](#second-bullet)

_for the moment, I return to the parent directory to be able to access the lips modules, this cell is not necessary if the lips package is installed in future_

In [7]:
import os
os.chdir(os.path.pardir)

In [9]:
from lips.benchmark import ConfigManager

2022-01-04 20:49:06.516600: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-01-04 20:49:06.516645: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Read an existing config <a class="anchor" id="first-bullet"></a>

Create an object of `ConfigManager`class. By indicating an existing benchmark name, its options will be restored by the config manager. You can also indicate a path to save your configuration, or set to `None` to use the default `config.ini` file.

In [10]:
cm = ConfigManager(benchmark_name="Benchmark1",
                   path=None
                  )

You can also, somehow, access all the benchmarks. See the available benchmarks (aka config sections)

In [12]:
cm.config.sections()

['Benchmark1', 'Benchmark2', 'Benchmark3']

#### Analyze the options available for a selected benchmark
These options are available for the first benchmark

In [14]:
cm.get_options_dict().keys()

dict_keys(['attr_names', 'theta_attr_names', 'eval_dict', 'utils_lib'])

#### Access the option values

In [15]:
cm.get_option("attr_names")

('prod_p',
 'prod_v',
 'load_p',
 'load_q',
 'line_status',
 'topo_vect',
 'a_or',
 'a_ex')

In [16]:
cm.get_option("theta_attr_names")

()

In [17]:
cm.get_option("eval_dict")

{'evaluate_ML': True,
 'evaluate_adaptability': False,
 'evaluate_readiness': False,
 'evaluate_physic': {'verify_current_pos': True,
  'verify_voltage_pos': False,
  'verify_loss_pos': False,
  'verify_predict_disc': True,
  'verify_current_eq': False,
  'verify_EL': False,
  'verify_LCE': False,
  'verify_KCL': False}}

#### Edit the options

In [19]:
cm.edit_config_option('theta_attr_names', "('theta_or', 'theta_ex')")

In [20]:
cm.get_option("theta_attr_names")

('theta_or', 'theta_ex')

#### Write this modification to `config.ini` file

In [25]:
cm._write_config()

## Create a new config for a new benchmark <a class="anchor" id="second-bullet"></a>

create an object of ConfigManager with a benchmark name that does not already exists.

In [26]:
cm = ConfigManager(benchmark_name="Benchmark4",
                   path=None
                  )

You can create the configuration by adding the required options as parameter. No restriction for the name of attributes at this step. 

**Nb :** However, these names will be used afterwards by `PowerGridBenchmark` class to parameterize the benchmark.

In [38]:
cm.create_config(attr_names="('p_or', 'a_or')")

add another option

In [69]:
description = "['this benchmark is intended to output power and current']"
cm.edit_config_option(option='description', value=description)

and remove an undesired option

In [72]:
cm.remove_config_option(option="description")

We can see that this benchmark is added among the existing benchmarks.

In [64]:
cm.config.sections()

['Benchmark1', 'Benchmark2', 'Benchmark3', 'Benchmark4']

We can verify its options.

In [73]:
cm.get_options_dict().keys()

dict_keys(['attr_names'])

And its values.

In [66]:
cm.get_option('attr_names')

('p_or', 'a_or')

And finally write it to the file.

In [74]:
cm._write_config()